# Datathon Week 1

Having the NYC problem we are going to explore multiple approaches on how to handle the data to answer some questions.

## Questions

 * What is the most used transportation method?
 * How uber behavior vs other transportation methods (in use)?
 * In which nta the people uses more uber?
 * 


In [1]:
import pandas as pd

In [5]:
uber_2014 = pd.read_csv('../Dataset/uber_trips_2014.csv')
uber_2014.head(10)

,pickup_datetime,pickup_latitude,pickup_longitude,base
0,4/1/14 0:11,40.7690,-73.9549,B02512
1,4/1/14 0:17,40.7267,-74.0345,B02512
2,4/1/14 0:21,40.7316,-73.9873,B02512
3,4/1/14 0:28,40.7588,-73.9776,B02512
4,4/1/14 0:33,40.7594,-73.9722,B02512
5,4/1/14 0:33,40.7383,-74.0403,B02512
6,4/1/14 0:39,40.7223,-73.9887,B02512
7,4/1/14 0:45,40.7620,-73.9790,B02512
8,4/1/14 0:55,40.7524,-73.9960,B02512
9,4/1/14 1:01,40.7575,-73.9846,B02512


In [8]:
uber_2015 = pd.read_csv('../Dataset/uber_trips_2015.csv')
uber_2015.head(10)

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base
0,2015-05-17 09:47:00,141,B02617,B02617
1,2015-05-17 09:47:00,65,B02617,B02617
2,2015-05-17 09:47:00,100,B02617,B02617
3,2015-05-17 09:47:00,80,B02617,B02774
4,2015-05-17 09:47:00,90,B02617,B02617
5,2015-05-17 09:47:00,228,B02617,B02617
6,2015-05-17 09:47:00,7,B02617,B02617
7,2015-05-17 09:47:00,74,B02617,B02764
8,2015-05-17 09:47:00,249,B02617,B02617
9,2015-05-17 09:47:00,22,B02617,B02764


In [9]:
yellow_cabs = pd.read_csv('../Dataset/yellow_trips.csv')
yellow_cabs.head(10)

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount
0,2014-04-03 18:28:10,2014-04-03 18:54:32,-74.006033,40.706284,-73.918837,40.744946,1,8.70,34.80
1,2014-04-16 15:42:00,2014-04-16 16:07:00,-73.979558,40.749357,0.000000,0.000000,1,9.37,30.00
2,2014-04-13 18:04:00,2014-04-13 18:07:00,-73.956453,40.775307,-73.954792,40.784992,1,0.89,6.50
3,2014-05-21 19:33:00,2014-05-21 19:45:00,-73.987212,40.757850,-73.960198,40.775472,1,2.40,12.50
4,2014-05-30 16:28:00,2014-05-30 16:51:00,-73.974292,40.755397,-74.011867,40.704222,3,5.58,24.50
5,2014-05-11 13:09:24,2014-05-11 13:19:18,-73.978131,40.748238,-73.984475,40.749268,1,1.30,11.05
6,2014-05-05 16:46:00,2014-05-05 16:49:00,-73.994868,40.745097,-74.004258,40.731687,2,1.10,8.30
7,2014-05-19 01:24:51,2014-05-19 01:37:11,-73.974711,40.764945,-73.916280,40.766681,1,4.00,15.50
8,2014-05-26 01:12:14,2014-05-26 01:34:44,-73.988324,40.723345,-73.953039,40.809264,1,7.50,24.50
9,2014-05-09 18:45:09,2014-05-09 18:57:12,-73.969744,40.753129,-73.988274,40.727640,1,2.00,13.80


In [10]:
green_cabs = pd.read_csv('../Dataset/green_trips.csv')
green_cabs.head(10)

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.80
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.80
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.30
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.30
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.50
5,2015-03-04 21:36:48,2015-03-04 21:43:47,-73.954826,40.804962,-73.944008,40.812088,1,1.16,9.36
6,2015-01-21 09:27:41,2015-01-21 09:51:01,-73.862000,40.730438,-73.865089,40.656422,1,7.50,25.80
7,2015-03-07 18:51:58,2015-03-07 19:20:49,-73.971947,40.675697,-74.009293,40.635708,6,4.58,21.30
8,2015-01-11 16:55:04,2015-01-11 17:04:26,-73.949883,40.681896,-73.923279,40.684875,1,1.51,8.80
9,2014-05-30 05:53:15,2014-05-30 06:00:00,-73.952370,40.789875,-73.949173,40.781448,1,1.20,7.50


In [11]:
metro = pd.read_csv('../Dataset/mta_trips.csv')
metro.head(10)

/Users/juanprq/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,station,line_name,division,audit_type,unit_id,datetime,new_entries,new_exits,latitude,longitude
0,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/29/2014 05:00:00,4,6,40.703087,-74.012994
1,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/29/2014 09:00:00,1,13,40.703087,-74.012994
2,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/29/2014 13:00:00,1,8,40.703087,-74.012994
3,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/29/2014 17:00:00,1,8,40.703087,-74.012994
4,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/29/2014 21:00:00,2,3,40.703087,-74.012994
5,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/30/2014 01:00:00,1,6,40.703087,-74.012994
6,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/30/2014 09:00:00,1,3,40.703087,-74.012994
7,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/30/2014 13:00:00,3,4,40.703087,-74.012994
8,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/30/2014 21:00:00,1,7,40.703087,-74.012994
9,WHITEHALL ST,R1,BMT,REGULAR,R001_A058_01-00-00,03/31/2014 01:00:00,1,3,40.703087,-74.012994


In [13]:
# uber_2014['datetime'] = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
uber_2014['datetime'] = uber_2014['pickup_datetime'].astype('datetime64[ns]')
uber_2014.head(10)

,pickup_datetime,pickup_latitude,pickup_longitude,base,datetime
0,4/1/14 0:11,40.7690,-73.9549,B02512,2014-04-01 00:11:00
1,4/1/14 0:17,40.7267,-74.0345,B02512,2014-04-01 00:17:00
2,4/1/14 0:21,40.7316,-73.9873,B02512,2014-04-01 00:21:00
3,4/1/14 0:28,40.7588,-73.9776,B02512,2014-04-01 00:28:00
4,4/1/14 0:33,40.7594,-73.9722,B02512,2014-04-01 00:33:00
5,4/1/14 0:33,40.7383,-74.0403,B02512,2014-04-01 00:33:00
6,4/1/14 0:39,40.7223,-73.9887,B02512,2014-04-01 00:39:00
7,4/1/14 0:45,40.7620,-73.9790,B02512,2014-04-01 00:45:00
8,4/1/14 0:55,40.7524,-73.9960,B02512,2014-04-01 00:55:00
9,4/1/14 1:01,40.7575,-73.9846,B02512,2014-04-01 01:01:00


In [ ]:
df.groupby([df['datetime'].dt.date]).count()